In [32]:
import sqlite3

conn = sqlite3.connect('Cell_inventory.db')
cur = conn.cursor()


In [33]:
from sqlalchemy import create_engine
import pandas as pd

URL = "CONNECTION STRING"
engine = create_engine(URL, echo=True)

In [34]:
df_test_inventory = pd.read_csv('csv/cell_test_inventory.csv')
df_test_inventory.keys()

Index(['Cell Type ID', 'Sample ID', 'Serial Number (Cell)', 'Customer',
       'Date Received (DD/MM/YYYY)', 'Purpose', 'Cell Producer', 'Cell Model',
       'Nominal Cell Capacity (Ah)', 'Form Factor',
       'Max rated current (charge) (A)', 'Max rated current (discharge) (A)',
       'Voltage (V)', 'AC-IR (mOhm) - 1 kHz', 'Weight (g)', 'Dimensions',
       'Battery chemistry', 'Cathode chemistry', 'Anode chemistry',
       'Additional tags (comma - separated)', 'Test Name', 'Cycler', 'Chamber',
       'Channel', 'Test ID', 'Test Procedure', 'Temperature (C)', 'SoC',
       'Test Current (A)', 'Comment', 'LEGACY testprocedure', 'Unnamed: 31',
       'Storage Cabinet', 'Cabinet Shelf'],
      dtype='object')

In [35]:
rename_dict = {
      'Cell Type ID': 'type_id',
      'Sample ID': 'sample_id',
      'Serial Number (Cell)': 'serial_number',
      'Customer': 'customer_id',
      'Date Received (DD/MM/YYYY)': 'date_received',
      'Purpose': 'purpose',
      'Cell Producer': 'cell_producer_id',
      'Cell Model': 'cell_model_id',
      'Nominal Cell Capacity (Ah)': 'nominal_capacity',
      'Form Factor': 'form_factor_id',
      'Max rated current (charge) (A)': 'charge_max_rated_current',
      'Max rated current (discharge) (A)': 'discharge_max_rated_current',
        'Voltage (V)': 'voltage',
        'AC-IR (mOhm) - 1 kHz': 'ac_ir',
        'Weight (g)': 'weight',
        'Dimensions': 'dimensions',
        'Battery chemistry': 'battery_chemistry_id',
        'Cathode chemistry': 'cathode_chemistry_id',
        'Anode chemistry': 'anode_chemistry_id',
        'Additional tags (comma - separated)': 'additional_tags',
        'Test Name': 'test_name',
        'Cycler': 'cycler',
        'Test Procedure': 'procedure',
        'Temperature (C)': 'temperature',
        'SoC': 'soc',
        'Test Current (A)': 'test_current',
        'Comment': 'comment',
        'LEGACY testprocedure': 'legacy_testprocedure',
        'Storage Cabinet': 'storage_location',
        'Cabinet Shelf': 'cabinet_shelf',
        'Test ID': 'test_id',
        'Chamber': 'chamber',
        'Channel': 'channel',
    }



df_test_inventory.rename(columns=rename_dict, inplace=True)
df_append_sqlite = df_test_inventory.copy()


In [36]:
df_test_inventory.drop(columns=['soc', 'test_current','cycler', 'legacy_testprocedure', 'Unnamed: 31', 'cabinet_shelf', 'sample_id', 'test_name', 'comment', 'temperature', 'procedure', 'test_id', 'chamber', 'channel'], inplace=True)

In [37]:
df_test_inventory['date_received'] = pd.to_datetime(df_test_inventory['date_received'], format='mixed').dt.strftime('%Y-%m-%d')

In [38]:
len(df_test_inventory.keys())
df_test_inventory.keys()

Index(['type_id', 'serial_number', 'customer_id', 'date_received', 'purpose',
       'cell_producer_id', 'cell_model_id', 'nominal_capacity',
       'form_factor_id', 'charge_max_rated_current',
       'discharge_max_rated_current', 'voltage', 'ac_ir', 'weight',
       'dimensions', 'battery_chemistry_id', 'cathode_chemistry_id',
       'anode_chemistry_id', 'additional_tags', 'storage_location'],
      dtype='object')

In [39]:
df_cell_info = pd.read_sql_query("SELECT cell_type_id AS type_id, cell_number, serial_number, date_received, date_released, purpose, nominal_capacity, max_rated_current_charge AS charge_max_rated_current, max_rated_current_discharge AS discharge_max_rated_current, voltage, ac_ir, weight, dimensions, additional_tags, storage_location, customer AS customer_id, battery_chemistry AS battery_chemistry_id, anode_chemistry as anode_chemistry_id, cathode_chemistry AS cathode_chemistry_id, cell_model AS cell_model_id, form_factor AS form_factor_id, cell_producer AS cell_producer_id, responsible_person AS responsible_person_id from table_cells", conn)



In [40]:
df_cell_info.keys()

Index(['type_id', 'cell_number', 'serial_number', 'date_received',
       'date_released', 'purpose', 'nominal_capacity',
       'charge_max_rated_current', 'discharge_max_rated_current', 'voltage',
       'ac_ir', 'weight', 'dimensions', 'additional_tags', 'storage_location',
       'customer_id', 'battery_chemistry_id', 'anode_chemistry_id',
       'cathode_chemistry_id', 'cell_model_id', 'form_factor_id',
       'cell_producer_id', 'responsible_person_id'],
      dtype='object')

In [41]:
df_cell_info = pd.concat([df_cell_info, df_test_inventory], axis=0, ignore_index=True)
df_cell_info.dropna(subset=['type_id', 'cell_number'], how='all', inplace=True)
df_cell_info

,type_id,cell_number,serial_number,date_received,date_released,purpose,nominal_capacity,charge_max_rated_current,discharge_max_rated_current,voltage,...,additional_tags,storage_location,customer_id,battery_chemistry_id,anode_chemistry_id,cathode_chemistry_id,cell_model_id,form_factor_id,cell_producer_id,responsible_person_id
0,EV_SKE_556,069,EV_SKE556_pouch,2024-04-12,None,Aging,57.5,,,3.6138,...,,3,Electra Vehicles,Li-ion,Graphite,NMC,556,Pouch,SK Energy,Shuo
1,BMW_LTF_2795,002,,2025-02-03,None,,35,,,,...,,,BMW,,,,,Cylindrical,,
2,BMW_LTF_2795,001,,2025-02-03,None,,35,,,,...,,,BMW,,,,,Cylindrical,,
3,BMW_LTF_3012,001,,2025-02-03,None,,33,,,,...,,,BMW,,,,,Cylindrical,,
4,BMW_LTF_2056,003,,2025-02-03,None,,33,,,,...,,,BMW,,,,,Cylindrical,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6142,LTF-2066,NaN,2023102612300312027,NaN,NaN,NaN,NaN,NaN,NaN,3.502,...,NaN,NaN,BMW,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6143,LTF-2072,NaN,2023101712300275010,NaN,NaN,NaN,NaN,NaN,NaN,3.511,...,NaN,NaN,BMW,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6144,LTF-2072,NaN,2023101712300275011,NaN,NaN,NaN,NaN,NaN,NaN,3.511,...,NaN,NaN,BMW,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6145,LTF-2072,NaN,2023101712300275010,NaN,NaN,NaN,NaN,NaN,NaN,3.511,...,NaN,NaN,BMW,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
df_cell_info['storage_location'].replace('c', 4, inplace=True)

/var/folders/yl/rlgr0xh932b142b9h7nkk8x80000gn/T/ipykernel_40506/564337855.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cell_info['storage_location'].replace('c', 4, inplace=True)


In [43]:
df_costumers = pd.DataFrame(df_cell_info['customer_id'].unique(), columns=['name'])
df_battery_chemistry = pd.DataFrame(df_cell_info['battery_chemistry_id'].unique(), columns=['name'])
df_anode_chemistry = pd.DataFrame(df_cell_info['anode_chemistry_id'].unique(), columns=['name'])
df_cathode_chemistry = pd.DataFrame(df_cell_info['cathode_chemistry_id'].unique(), columns=['name'])
df_cell_model = pd.DataFrame(df_cell_info['cell_model_id'].unique(), columns=['name'])
df_form_factor = pd.DataFrame(df_cell_info['form_factor_id'].unique(), columns=['name'])
df_producer = pd.DataFrame(df_cell_info['cell_producer_id'].unique(), columns=['name'])
df_test_engineer = pd.DataFrame(df_cell_info['responsible_person_id'].unique(), columns=['name'])
df_test_procedure = pd.DataFrame(df_cell_info['purpose'].unique(), columns=['name'])

df_costumers

,name
0,Electra Vehicles
1,BMW
2,Internal
3,Daimler Truck
4,CustomCells
5,IVECO
6,Sphere
7,NaN
8,Festool
9,Custom Cells


In [44]:
df_test_inventory.keys()

Index(['type_id', 'serial_number', 'customer_id', 'date_received', 'purpose',
       'cell_producer_id', 'cell_model_id', 'nominal_capacity',
       'form_factor_id', 'charge_max_rated_current',
       'discharge_max_rated_current', 'voltage', 'ac_ir', 'weight',
       'dimensions', 'battery_chemistry_id', 'cathode_chemistry_id',
       'anode_chemistry_id', 'additional_tags', 'storage_location'],
      dtype='object')

In [45]:
df_battery_chemistry.to_sql('BATTERY_CHEMISTRY', con=engine, if_exists='append', index=False)
df_anode_chemistry.to_sql('ANODE_CHEMISTRY', con=engine, if_exists='append', index=False)
df_cathode_chemistry.to_sql('CATHODE_CHEMISTRY', con=engine, if_exists='append', index=False)
df_cell_model.to_sql('CELL_MODEL', con=engine, if_exists='append', index=False)
df_form_factor.to_sql('FORM_FACTOR', con=engine, if_exists='append', index=False)
df_producer.to_sql('CELL_PRODUCER', con=engine, if_exists='append', index=False)
df_test_engineer.to_sql('TEST_ENGINEER', con=engine, if_exists='append', index=False)
df_costumers.to_sql('CUSTOMER', con=engine, if_exists='append', index=False)

2025-03-06 12:00:52,354 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-03-06 12:00:52,355 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-06 12:00:52,382 INFO sqlalchemy.engine.Engine select current_schema()
2025-03-06 12:00:52,382 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-06 12:00:52,423 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-03-06 12:00:52,424 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-06 12:00:52,448 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-06 12:00:52,454 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

15

In [46]:
df_test_procedure.to_sql('TEST_PROCEDURE', con=engine, if_exists='append', index=False)

2025-03-06 12:00:53,007 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-06 12:00:53,009 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-03-06 12:00:53,009 INFO sqlalchemy.engine.Engine [cached since 0.5553s ago] {'table_name': 'TEST_PROCEDURE', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-03-06 12:00:53,039 INFO sqlalchemy.engine.Engine INSERT INTO "TEST_PROCEDURE" (name) VALUES (%(name__0)s), (%(name__1)s), (%(name__2)s), (%(name__3)s), (%(name__4)s), (%(name__5)s), (%(name__6)s), (%(name__7)s), (%(na

16

In [47]:
df_costumers.reset_index(inplace=True)
df_cell_info = df_cell_info.merge(df_costumers, left_on='customer_id', right_on='name', how='left')
df_cell_info['customer_id'] = df_cell_info['index'] + 1
df_cell_info.drop(columns=['index'], inplace=True)


In [48]:
df_battery_chemistry.reset_index(inplace=True)
df_cell_info = df_cell_info.merge(df_battery_chemistry, left_on='battery_chemistry_id', right_on='name', how='left')
df_cell_info['battery_chemistry_id'] = df_cell_info['index'] + 1
df_cell_info.drop(columns=['index'], inplace=True)
df_cell_info.drop(columns=['name_y'], inplace=True)

df_anode_chemistry.reset_index(inplace=True)
df_cell_info = df_cell_info.merge(df_anode_chemistry, left_on='anode_chemistry_id', right_on='name', how='left')
df_cell_info['anode_chemistry_id'] = df_cell_info['index'] + 1
df_cell_info.drop(columns=['index'], inplace=True)
df_cell_info.drop(columns=['name_x'], inplace=True)

df_cathode_chemistry.reset_index(inplace=True)
df_cell_info = df_cell_info.merge(df_cathode_chemistry, left_on='cathode_chemistry_id', right_on='name', how='left', suffixes=('', '_cc'))
df_cell_info['cathode_chemistry_id'] = df_cell_info['index'] + 1
df_cell_info.drop(columns=['index'], inplace=True)
df_cell_info.drop(columns=['name_cc'], inplace=True)

df_cell_model.reset_index(inplace=True)
df_cell_info = df_cell_info.merge(df_cell_model, left_on='cell_model_id', right_on='name', how='left', suffixes=('', '_cm'))
df_cell_info['cell_model_id'] = df_cell_info['index'] + 1
df_cell_info.drop(columns=['index'], inplace=True)
df_cell_info.drop(columns=['name_cm'], inplace=True)

df_form_factor.reset_index(inplace=True)
df_cell_info = df_cell_info.merge(df_form_factor, left_on='form_factor_id', right_on='name', how='left', suffixes=('', '_ff'))
df_cell_info['form_factor_id'] = df_cell_info['index'] + 1
df_cell_info.drop(columns=['index'], inplace=True)
df_cell_info.drop(columns=['name_ff'], inplace=True)

df_producer.reset_index(inplace=True)
df_cell_info = df_cell_info.merge(df_producer, left_on='cell_producer_id', right_on='name', how='left', suffixes=('', '_cp'))
df_cell_info['cell_producer_id'] = df_cell_info['index'] + 1
df_cell_info.drop(columns=['index'], inplace=True)
df_cell_info.drop(columns=['name_cp'], inplace=True)

df_test_engineer.reset_index(inplace=True)
df_cell_info = df_cell_info.merge(df_test_engineer, left_on='responsible_person_id', right_on='name', how='left', suffixes=('', '_rp'))
df_cell_info['responsible_person_id'] = df_cell_info['index'] + 1
df_cell_info.drop(columns=['index'], inplace=True)
df_cell_info.drop(columns=['name_rp'], inplace=True)

df_cell_info.drop(columns=['name'], inplace=True)

df_cell_info.replace('None', None, inplace=True)
df_cell_info.replace('', None, inplace=True)
df_cell_info


,type_id,cell_number,serial_number,date_received,date_released,purpose,nominal_capacity,charge_max_rated_current,discharge_max_rated_current,voltage,...,additional_tags,storage_location,customer_id,battery_chemistry_id,anode_chemistry_id,cathode_chemistry_id,cell_model_id,form_factor_id,cell_producer_id,responsible_person_id
0,EV_SKE_556,069,EV_SKE556_pouch,2024-04-12,None,Aging,57.5,None,None,3.6138,...,None,3,1,1,1,1,1,1,1,1
1,BMW_LTF_2795,002,None,2025-02-03,None,None,35,None,None,None,...,None,None,2,2,2,2,2,2,2,2
2,BMW_LTF_2795,001,None,2025-02-03,None,None,35,None,None,None,...,None,None,2,2,2,2,2,2,2,2
3,BMW_LTF_3012,001,None,2025-02-03,None,None,33,None,None,None,...,None,None,2,2,2,2,2,2,2,2
4,BMW_LTF_2056,003,None,2025-02-03,None,None,33,None,None,None,...,None,None,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4512,LTF-2066,NaN,2023102612300312027,NaN,NaN,NaN,NaN,NaN,NaN,3.502,...,NaN,NaN,2,5,3,4,13,4,15,9
4513,LTF-2072,NaN,2023101712300275010,NaN,NaN,NaN,NaN,NaN,NaN,3.511,...,NaN,NaN,2,5,3,4,13,4,15,9
4514,LTF-2072,NaN,2023101712300275011,NaN,NaN,NaN,NaN,NaN,NaN,3.511,...,NaN,NaN,2,5,3,4,13,4,15,9
4515,LTF-2072,NaN,2023101712300275010,NaN,NaN,NaN,NaN,NaN,NaN,3.511,...,NaN,NaN,2,5,3,4,13,4,15,9


In [49]:
df_cell_info.drop(columns=['form_factor_id'], inplace=True)
df_cell_info.to_sql('CELL_INFO', con=engine, if_exists='append', index=False)

2025-03-06 12:00:53,161 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-06 12:00:53,163 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-03-06 12:00:53,164 INFO sqlalchemy.engine.Engine [cached since 0.7102s ago] {'table_name': 'CELL_INFO', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-03-06 12:00:53,234 INFO sqlalchemy.engine.Engine INSERT INTO "CELL_INFO" (type_id, cell_number, serial_number, date_received, date_released, purpose, nominal_capacity, charge_max_rated_current, discharge_max_rated_current,

517

In [50]:
import datetime

cycler_dict = {
    "serial_number": ["229383-A", "229383-A", "229383-B", "229383-C", "33247", "234783-C", "234783-B", "234783-A1", "234783-A2", "234775"],
    "active_from": [None, datetime.datetime.fromtimestamp(1734429154), None, None, None, None, None, None, None, None],
    "short_name": ["A1", "A1-1", "A2", "A3", "A4", "A5", "A6", "A7a", "A7b", "A8"],
    "long_name": ["Arbin1", "Arbin1-1", "Arbin2", "Arbin3", "Arbin4", "Arbin5", "Arbin6", "Arbin7a", "Arbin7b", "Arbin8"]
}



df_cycler = pd.DataFrame(cycler_dict)
df_cycler.to_sql(name='CYCLER_INFO', con=engine, if_exists='append', index=False)



2025-03-06 12:00:54,603 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-06 12:00:54,604 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-03-06 12:00:54,605 INFO sqlalchemy.engine.Engine [cached since 2.151s ago] {'table_name': 'CYCLER_INFO', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-03-06 12:00:54,629 INFO sqlalchemy.engine.Engine INSERT INTO "CYCLER_INFO" (serial_number, active_from, short_name, long_name) VALUES (%(serial_number__0)s, %(active_from__0)s, %(short_name__0)s, %(long_name__0)s), (%(ser

10

# Ingest the old test spreadsheet

In [51]:
# df_append_sqlite.drop(columns=['type_id', 'charge_max_rated_current', 'discharge_max_rated_current','voltage', 'ac_ir','weight', 'dimensions','battery_chemistry_id', 'cathode_chemistry_id', 'anode_chemistry_id', 'additional_tags','purpose','date_received', 'cell_producer_id', 'nominal_capacity', 'cell_model_id', 'form_factor_id','test_current','serial_number', 'customer_id', 'legacy_testprocedure', 'Unnamed: 31', 'cabinet_shelf', 'comment', 'soc', 'storage_location'], inplace=True)

In [52]:
# df_append_sqlite['test_key'] = f"{df_append_sqlite['cycler']}_{df_append_sqlite['channel']}_{df_append_sqlite['test_id']}"
# df_append_sqlite.keys()


In [53]:
# df_append_sqlite.to_sql(name='table_tests', con=conn, if_exists='append', index=False)

In [54]:
# count = pd.read_sql_query("SELECT COUNT(*) FROM table_tests", conn)
# count

In [55]:
# pd.read_sql_query("SELECT * FROM table_tests WHERE cycler='A1' ORDER BY test_id ASC", conn)